<a href="https://colab.research.google.com/github/SELOMELO280104/Roboflow_test/blob/main/polygon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proje Başlatıldı

Polygon Test

In [ ]:
# Gerekli kütüphaneleri kur
!pip install ultralytics -q
import os
import zipfile
from google.colab import files
from ultralytics import YOLO

# 1. Zip dosyasını yükleme ekranı
print("👇 Lütfen Roboflow'dan indirdiğiniz POLYGON (Segmentation) '.zip' veri setini seçin:")
uploaded = files.upload()

if uploaded:
    zip_filename = list(uploaded.keys())[0]
    extract_dir = '/content/dataset_seg' # Klasör adını seg olarak güncelledik

    # 2. Zip dosyasını çıkartma
    print(f"\n📦 '{zip_filename}' çıkartılıyor...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    # 3. data.yaml dosyasını otomatik bulma
    yaml_path = None
    for root, dirs, dosyalar in os.walk(extract_dir):
        if 'data.yaml' in dosyalar:
            yaml_path = os.path.join(root, 'data.yaml')
            break

    if yaml_path:
        print(f"✅ Polygon veri seti hazır! Dosya yolu: {yaml_path}")

        # 4. Eğitimi Başlatma (SEGMENTASYON İÇİN -seg takısı şart)
        print("\n🚀 Polygon (Segmentasyon) eğitimi T4 GPU üzerinde başlıyor...")

        # KRİTİK DEĞİŞİKLİK: yolov8n-seg.pt kullanıyoruz
        model = YOLO('yolov8n-seg.pt')

        results = model.train(
            data=yaml_path,
            epochs=30,
            imgsz=640,
            batch=16,
            project='/content/Trained_model_seg', # Proje klasör adını ayırdık
            name='ayakkabi_polygon_sonuc'
        )
        print("\n🎉 Polygon eğitimi başarıyla tamamlandı!")
    else:
        print("❌ HATA: Zip dosyasının içinde 'data.yaml' bulunamadı!")
else:
    print("❌ Dosya yüklemesi iptal edildi.")

In [ ]:
from IPython.display import Image, display
import glob
from ultralytics import YOLO

print("🔍 Polygon (Segmentasyon) test verileriyle performans ölçülüyor...")

# 1. Model yolunu kontrol et (Sol menüden doğrula)
best_model_path = '/content/Trained_model_seg/ayakkabi_polygon_sonuc/weights/best.pt'
model = YOLO(best_model_path)

# 2. Doğrulama yaparken conf parametresini ekliyoruz
# conf=0.25 yaparsan sadece %25 ve üzeri emin olduğu maskeleri çizer
metrics = model.val(
    split='test',
    project='/content/Trained_model_seg',
    name='test_sonuclari_seg',
    conf=0.01,  # <--- BURADAN DÜŞÜREBİLİR VEYA YÜKSELTEBİLİRSİN
    iou=0.45    # Kesişim eşiği (opsiyonel)
)

# 3. Sonuçları göster
print("\n📷 Test Klasöründen Örnek Polygon Tespiti:")
resimler = glob.glob('/content/Trained_model_seg/test_sonuclari_seg*/val_batch0_pred.jpg')

if resimler:
    display(Image(filename=resimler[0]))
else:
    print("Örnek görsel bulunamadı.")

In [ ]:
from google.colab import files
import os

# Polygon projesi için doğru yol budur (Klasör ismini sol menüden teyit et)
# Eğer klasörün adı 'train' ise 'ayakkabi_polygon_sonuc' yerine 'train' yazmalısın.
best_model_path = '/content/Trained_model_seg/ayakkabi_polygon_sonuc/weights/best.pt'

if os.path.exists(best_model_path):
    print("⬇️ Eğitilmiş Polygon modeli (best.pt) indiriliyor...")
    files.download(best_model_path)
    print("✅ İndirme başlatıldı!")
else:
    print(f"❌ HATA: Dosya bulunamadı! Lütfen yolu kontrol et: {best_model_path}")
    print("💡 İpucu: Sol menüden 'weights' klasörüne git, best.pt dosyasına sağ tıkla ve 'Yolu kopyala' de.")